# Task 3

In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt 
import seaborn as sns

In [2]:
df = pd.read_csv('Task 3 and 4_Loan_Data.csv')

In [3]:
df.head(15)

,customer_id,credit_lines_outstanding,loan_amt_outstanding,total_debt_outstanding,income,years_employed,fico_score,default
0,8153374,0,5221.545193,3915.471226,78039.38546,5,605,0
1,7442532,5,1958.928726,8228.752520,26648.43525,2,572,1
2,2256073,0,3363.009259,2027.830850,65866.71246,4,602,0
3,4885975,0,4766.648001,2501.730397,74356.88347,5,612,0
4,4700614,1,1345.827718,1768.826187,23448.32631,6,631,0
5,4661159,0,5376.886873,7189.121298,85529.84591,2,697,0
6,8291909,1,3634.057471,7085.980095,68691.57707,6,722,0
7,4616950,4,3302.172238,13067.570210,50352.16821,3,545,1
8,3395789,0,2938.325123,1918.404472,53497.37754,4,676,0
9,4045948,0,5396.366774,5298.824524,92349.55399,2,447,0


In [4]:
df['default'].value_counts()

default
0    8149
1    1851
Name: count, dtype: int64

The class is highly imbalance so we need to addres it. We will perform oversampling using the SMOTE technique which is implemented after the data is split.

In [5]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
X = df.iloc[:,1:7]
y= df.iloc[:, 7:]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

#turn in to 1d arrays
y_train = y_train.to_numpy().ravel()
y_test = y_test.to_numpy().ravel()

oversample = SMOTE()
X_train_SMOTE, y_train_SMOTE = oversample.fit_resample(X_train, y_train)



In [6]:
from collections import Counter
counter = Counter(y_train_SMOTE)
for k,v in counter.items():
 per = v / len(y_train_SMOTE) * 100
 print('Class=%d, n=%d (%.3f%%)' % (k, v, per))

Class=0, n=6089 (50.000%)
Class=1, n=6089 (50.000%)


In [7]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_SMOTE_scaled = pd.DataFrame(sc.fit_transform(X_train_SMOTE), columns= X_train_SMOTE.columns)

X_test_scaled = pd.DataFrame(sc.transform(X_test), columns=X_test.columns)

In [8]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train_SMOTE_scaled, y_train_SMOTE)

LogisticRegression()

In [9]:
import statsmodels.api as sm
logit_model=sm.Logit(y_train_SMOTE, X_train_SMOTE_scaled)
result=logit_model.fit()
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.040472
         Iterations 11
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                12178
Model:                          Logit   Df Residuals:                    12172
Method:                           MLE   Df Model:                            5
Date:                Tue, 12 Sep 2023   Pseudo R-squ.:                  0.9416
Time:                        05:38:10   Log-Likelihood:                -492.87
converged:                       True   LL-Null:                       -8441.1
Covariance Type:            nonrobust   LLR p-value:                     0.000
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
credit_lines_outstanding     5.3342      0.443     12.040      0.000       4.46

Most of the p-values are lower than 0.05 so there is a relation between the variables. However, `loan_amt_outstanding` has a p-value of 0.543 which implies that we cannot conclude there is a relation between `loan_amt_outstanding` and `default`. This means that there is a 54.3% chance that the true coefficient of `loan_amt_outstanding` is zero.

In [10]:
result.pvalues

credit_lines_outstanding    2.177778e-33
loan_amt_outstanding        5.427593e-01
total_debt_outstanding      1.041617e-13
income                      3.157707e-10
years_employed              1.630925e-46
fico_score                  5.835451e-15
dtype: float64

In [11]:
from sklearn.metrics import classification_report
lr_predictions = lr.predict(X_train_SMOTE_scaled)
print(classification_report(y_train_SMOTE, lr_predictions, zero_division=0))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99      6089
           1       0.99      1.00      0.99      6089

    accuracy                           0.99     12178
   macro avg       0.99      0.99      0.99     12178
weighted avg       0.99      0.99      0.99     12178



A cross validation of the logistic regression shows a baseline of 99.33% which is the same to the one obtained when we train the data above. After predicting the test data we get 99 accuracy which indicates that our model is performing similar when it is applied to other data sets.

In [12]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
kfold = KFold(n_splits=5, shuffle=True, random_state=0)
results_lr = cross_val_score(lr, X_train_SMOTE_scaled, y_train_SMOTE, cv=kfold, scoring='accuracy')
print(results_lr)
print("Baseline: %.2f%% (%.2f%%)" % (results_lr.mean()*100, results_lr.std()*100))

[0.98973727 0.9954844  0.99178982 0.99712526 0.9926078 ]
Baseline: 99.33% (0.26%)


In [13]:
lr_predictions_test = lr.predict(X_test_scaled)
print(classification_report(y_test, lr_predictions_test, zero_division=0))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00      2060
           1       0.96      1.00      0.98       440

    accuracy                           0.99      2500
   macro avg       0.98      1.00      0.99      2500
weighted avg       0.99      0.99      0.99      2500



The precision of class 1 tells us that 96% of the bad loan borrowers that our model indentified were actually bad loan borrowers.

The recall of class 1 tells us that 100% bad loan borrowers out of all the bad loan borrowers in the test set. 

Finally, we need to add the probability to default to each of the customers.

In [20]:
X_scaled = pd.DataFrame(sc.transform(X), columns=X.columns)

In [23]:
df['PD'] = lr.predict_proba(X_scaled)[:,1]
df[['customer_id', 'PD']].head(15)

,customer_id,PD
0,8153374,1.874574e-11
1,7442532,9.999999e-01
2,2256073,1.530242e-10
3,4885975,8.668466e-12
4,4700614,2.177360e-07
5,4661159,3.116352e-09
6,8291909,1.375121e-09
7,4616950,9.999738e-01
8,3395789,2.683346e-10
9,4045948,2.914193e-08


It is worth to mention that we do a scaling of all the features when it was not required for all them. We performed the scaling after splitting the data to avoid leaking information of the test set. However, this makes difficult to predict new information without being rescaled first. One can omit the scaling and repeat all the steps perhaps at the cost of increasing the value of max_iter in our LogisticRegression model.